In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/poetry")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/poetry


Step 1: Install Hugging Face Transformers

In [4]:
!pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-c

Imports

In [5]:
import pandas as pd
from transformers import (
    GPT2LMHeadModel, GPT2Tokenizer,
    TextDataset, DataCollatorForLanguageModeling,
    Trainer, TrainingArguments, pipeline, set_seed
)
import os

In [6]:
import os

dataset_path = "/kaggle/input/poetry"
output_file = "poems.txt"

# Combine all text files into one
with open(output_file, "w", encoding="utf-8") as outfile:
    for filename in os.listdir(dataset_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(dataset_path, filename)
            with open(file_path, "r", encoding="utf-8") as infile:
                content = infile.read().strip()
                outfile.write(content + "\n\n")

print(f"✅ Combined all artist lyrics into: {output_file}")

✅ Combined all artist lyrics into: poems.txt


In [7]:
dataset_path = "/kaggle/input/poetry"
print("Files in dataset folder:")
print(os.listdir(dataset_path))

Files in dataset folder:
['Kanye_West.txt', 'johnny-cash.txt', 'kanye-west.txt', 'bruno-mars.txt', 'dickinson.txt', 'amy-winehouse.txt', 'blink-182.txt', 'paul-simon.txt', 'patti-smith.txt', 'bieber.txt', 'disney.txt', 'jimi-hendrix.txt', 'lin-manuel-miranda.txt', 'adele.txt', 'dj-khaled.txt', 'beatles.txt', 'r-kelly.txt', 'lady-gaga.txt', 'radiohead.txt', 'britney-spears.txt', 'alicia-keys.txt', 'rihanna.txt', 'joni-mitchell.txt', 'dolly-parton.txt', 'drake.txt', 'Lil_Wayne.txt', 'notorious_big.txt', 'eminem.txt', 'janisjoplin.txt', 'prince.txt', 'bruce-springsteen.txt', 'bob-dylan.txt', 'notorious-big.txt', 'lil-wayne.txt', 'dr-seuss.txt', 'nicki-minaj.txt', 'bob-marley.txt', 'al-green.txt', 'nickelback.txt', 'michael-jackson.txt', 'lorde.txt', 'kanye.txt', 'leonard-cohen.txt', 'ludacris.txt', 'bjork.txt', 'nursery_rhymes.txt', 'nirvana.txt', 'cake.txt', 'missy-elliott.txt']


Load tokenizer and model

In [8]:
# 🔹 Step 5: Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Required by Trainer
model = GPT2LMHeadModel.from_pretrained("gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]